# Очистка данных и регулярные выражения

Очистка данных — первый этап обработки информации (после её получения), который позволяет:

- Удалить шум (лишние символы, HTML-теги, пробелы, стоп-слова)
- Привести текст к единому формату (нижний регистр, единая кодировка)
- Исправить ошибки (убрать дубликаты, заменить символы)
- Подготовить данные для анализа или хранения

Для некоторых проблем подходят простые и уже знакомые нам решения

In [117]:
bad_text = '   А мой   мальчик едет    на девятке\nПо автостраде вдоль ночных дорог\n   '
bad_text

'   А мой   мальчик едет    на девятке\nПо автостраде вдоль ночных дорог\n   '

Например, в строке выше:
- лишние обрамляющие пробелы
- лишние пробелы внутри
- внутренние избыточные символы переноса

В данном случае можно обойтись встроенными методами строк

Например, так:

In [323]:
# split будет разделять по пробельным символам и в том числе символам \n переноса, если разделитель не передан
# пустые строки будут удалены

good_text = bad_text.strip().split("\n")  
print(good_text)
good_text = [" ".join(i.split()) for i in good_text]
print(good_text)

['А мой   мальчик едет    на девятке', 'По автостраде вдоль ночных дорог']
['А мой мальчик едет на девятке', 'По автостраде вдоль ночных дорог']


Однако некоторые проблемы сложно эффективно решить через методы строк

Например:
- Удаление HTML-тегов
- Обработка специальных символов из содержания веб-страницы после первичного парсинга
- Мэтчинг номеров телефонов внутри текста в соответствии с заданным форматом

## Простые проблемы и их решения

### Удаление пробелов и специальных символов 

Здесь часто подойдут уже знакомые нам методы строк

__Обрезка пробельных символов:__

In [45]:
text = "   Пример текста   "
print(text.strip())   # "Пример текста"
print(text.lstrip())  # "Пример текста   "
print(text.rstrip())  # "   Пример текста"

Пример текста
Пример текста   
   Пример текста


__Удаление отдельных символов:__

In [47]:
text = "hello@world@example.com"
print(text.replace("@", " "))  # "hello world example.com"

hello world example.com


__Удаление пунктуации:__

In [334]:
import string

text = "Привет, мир! Как дела?"
clean_text = text.translate(str.maketrans("", "", string.punctuation))
print(clean_text)  # "Привет мир Как дела"

Привет мир Как дела


`str.maketrans` создаёт словарь, согласно которому в строке будут произведены замены ключей на его значения

Например, пара `{33: None}` означает, что символ `chr(33)` (\u0021) будет удалён (символ ! -> None)

In [335]:
# для перевода чисел-ключей в кодпоинт юникод достаточно перевести числа-ключи в шестнадцатиричную систему:
symbol = "!"
unicode_codpoint = f"\\u{ord(symbol):04x}"
print(unicode_codpoint)
symbol_back = unicode_codpoint.encode("utf-8").decode("unicode_escape")
print(symbol_back)

\u0021
!


### Удаление HTML-тегов

In [337]:
html_text = "<div><p>Привет, <a href='some_link.com'>друзья</a>!</p></div>"

Как правило это уже знакомый нам интерфейс bs4 (например)

In [338]:
from bs4 import BeautifulSoup

soup = BeautifulSoup(html_text, "html.parser")
clean_text = soup.get_text()
print(clean_text) 

Привет, друзья!


Иногда полезно делать замены html-кода, который лежит внутри нашего "супа" перед извлечением тегов/текста:

In [339]:
html_text = """
<span class="ReferentFragment-desktop__Highlight-sc-380d78dd-1 fIkrDi">А мой мальчик едет на девятке<br>По автостраде вдоль ночных дорог</span>
"""

soup = BeautifulSoup(html_text, "html.parser")
print(soup.prettify())
clean_text = soup.get_text()
print(clean_text)

<span class="ReferentFragment-desktop__Highlight-sc-380d78dd-1 fIkrDi">
 А мой мальчик едет на девятке
 <br/>
 По автостраде вдоль ночных дорог
</span>


А мой мальчик едет на девяткеПо автостраде вдоль ночных дорог



- Найдем тег, который мы хотим заменить (в данном случае это тег \<br>)
- Заменим его на `\n` (т.к. это аналог переноса строки)

In [342]:
br_tag = soup.find("br")
print(br_tag)

<br/>


In [343]:
br_tag.replace_with("\n")

<br/>

In [347]:
print(soup.prettify())

clean_text = soup.get_text()
print(clean_text)

<span class="ReferentFragment-desktop__Highlight-sc-380d78dd-1 fIkrDi">
 А мой мальчик едет на девятке
 По автостраде вдоль ночных дорог
</span>


А мой мальчик едет на девятке
По автостраде вдоль ночных дорог



### Удаление специальных юникод-символов и HTML-сущностей

При веб-скрэппинге могут появляться скрытые или неявные символы, которые не видны в DevTools (при просмотре HTML-кода в браузере или которые не отображаются "явно" при выводе строки через print), но появляются в обработанном тексте. Поэтому при парсинге веб-страниц мы иногда получаем что-то такое внутри извлеченного текста:
- \xa0
- или такое: \u2000
- или такое: \&lt;

И может быть не совсем понятно, что с этим делать, поскольку вроде бы кодировка веб-страницы указана верно, но всё равно что-то не так

#### Из наиболее частого встречается следующее

__1. Различные пробельные символы Unicode__:

| Escape-код  | Unicode  | Название                           |
|------------|---------|-----------------------------------|
| `\xa0`     | U+00A0  | Неразрывный пробел (NBSP) |
| `\u2000`   | U+2000  | En Quad (широкий пробел)          |
| `\u2001`   | U+2001  | Em Quad (ещё шире)                |
| `\u2002`   | U+2002  | En Space (как обычный пробел)     |

__2. Управляющие символы Unicode__:
| Escape-код  | Unicode  | Название                                        |
|------------|---------|------------------------------------------------|
| `\u200B`   | U+200B  | Zero Width Space (пробел нулевой ширины, используется для скрытия текста) |
| `\u200C`   | U+200C  | Zero Width Non-Joiner (разрывает лигатуры)     |
| `\u200D`   | U+200D  | Zero Width Joiner (соединяет эмодзи, например 👨‍👩‍👧‍👦) |

__3. Специальные кавычки и тире__:

| Escape-код  | Символ | Unicode  | Название                          |
|------------|--------|---------|----------------------------------|
| `\u2018`   | ‘      | U+2018  | Левая одинарная кавычка         |
| `\u2019`   | ’      | U+2019  | Правая одинарная кавычка (апостроф) |
| `\u201C`   | “      | U+201C  | Левые двойные кавычки           |

__4. HTML-сущности__:

| HTML      | Unicode  | Символ                                       |
|-----------|---------|---------------------------------------------|
| `&shy;`   | U+00AD  | Мягкий перенос (не отображается, но ломает текст) |
| `&copy;`  | U+00A9  | ©                               |
| `&reg;`   | U+00AE  | ®                              |
| `&nbsp;`  | U+00A0  | Неразрывный пробел (не ломает текст)|
| `&lt;`    | U+003C  | <                        |




- \u — это нотация для Unicode-символов. В Python формат \uXXXX используется для символов Unicode в диапазоне U+0000 – U+FFFF (16-битные символы) (или \UXXXXXXXX для 32-битных юникод символов, здесь *U* заглавная)
- \x - нотация для 8-битных символов. В Python, если символ попадает в диапазон 0–255, он может быть записан в формате \xXX
- &...; - нотация для [HTML-сущностей](https://www.w3schools.com/html/html_entities.asp)

Что делать, если перед нами строки, которые содержат что-то из этого в явном виде? 

Например:

In [348]:
bad_str_8bit = "Hello\xa0World!"
print(bad_str_8bit)

bad_str_unicode = "I love Python \u2764\uFE0F"
print(bad_str_unicode)

bad_str_html_entities = "Hello&nbsp;World&copy; 2025"
print(bad_str_html_entities)

Hello World!
I love Python ❤️
Hello&nbsp;World&copy; 2025


**Пример 1**: `Hello\xa0World!`

Здесь можно пойти двумя путями:

1) Оставить всё как есть (подходит, если строку мы будем использовать только для вывода)

In [349]:
print(bad_str_8bit)

Hello World!


2) Сделать замену на ближайший по смыслу символ (например, в данном случае - на пробел) или вовсе удалить

In [162]:
spaced_bad_str_8bit = bad_str_8bit.replace("\xa0", " ")
print(bad_str_8bit.find(" "))

print(spaced_bad_str_8bit)
print(spaced_bad_str_8bit.find(" "))

-1
Hello World!
5


Минус второго подхода в том, что существует не единственный пробельный символ в кодировке UTF-8:
- \xa0
- \x09
- \xad
- \x20
- и некоторые другие

То есть в общем случае надо пытаться делать замены на вообще все пробельные символы (и это только в кодировке UTF-8, и это только пробелы). В следующем разделе посмотрим, как решить эту проблему относительно просто.

А пока вот решение конкретно для пробельных символов

In [361]:
import unicodedata

print(unicodedata.category("\xa0"))
spaced_bad_str_8bit = ''.join(' ' if unicodedata.category(c) == 'Zs' else c for c in bad_str_8bit)
print(spaced_bad_str_8bit.find(" "))
print(spaced_bad_str_8bit)

Zs
5
Hello World!


Список категорий символов юникод можно посмотреть [здесь](https://www.fileformat.info/info/unicode/category/index.htm)

**Пример 2**: `I love Python \u2764\uFE0F` или `I\u2000love\u2000Python`

Здесь также есть несколько вариантов решений:

1. Оставить всё как есть (тогда вас скорее всего устраивает наличие юникод-символов в вашей строке)

In [195]:
print('I love Python \u2764\uFE0F')
print('I\u2000love\u2000Python')

I love Python ❤️
I love Python


2. Удалить все юникод-символы

Тут всё очень зависит от нашей цели.

- Либо мы идём по пути, например, поиска всех невыводимых (undisplayable) [категорий юникод-символов](https://www.fileformat.info/info/unicode/category/index.htm) (т.е. выборочное удаление). Здесь решение зависит от того, какие символы мы хотим убрать, и сводится к копированию логики выше с поиском через unicodedata.category
- Либо мы идем по пути [почти] полного удаления всех символов юникода, что на деле скорее означает удаление символов, которые не соответствуют какому-то алфавиту, числам и пунктуации. Решение для этого случая мы рассмотрим ниже (в блоке про регулярные выражения)

3. (для спец. символов) Привести символы юникод к некоторой единой форме (полезно для разных пробельных символов)

In [214]:
bad_str_unicode = 'I\u2000love\u2000Python'
print(bad_str_unicode)
print(bad_str_unicode.find(" "))

normalized_unicode = unicodedata.normalize('NFKD', 'I\u2000love\u2000Python')
print(normalized_unicode)
print(normalized_unicode.find(" "))

I love Python
-1
I love Python
1


**Пример 3**: `Hello&nbsp;World&copy; 2025`

In [224]:
import html

bad_str_html_entities = "Hello&nbsp;World&copy; 2025"


print(bad_str_html_entities)
decoded_str_html = html.unescape(bad_str_html_entities)
print(decoded_str_html)
print(decoded_str_html.find(" "))  # первый пробел - это символ \xa0; иногда это плохо
print("---")
decoded_str_html_whitespaced = ''.join(' ' if unicodedata.category(c) == 'Zs' else c for c in decoded_str_html)
print(decoded_str_html_whitespaced)
print(decoded_str_html_whitespaced.find(" "))  # теперь первый пробел - это пробельный символ " " (ему в восьмибитной нотации отвечает символ \x20)

Hello&nbsp;World&copy; 2025
Hello World© 2025
12
---
Hello World© 2025
5


## Более сложные проблемы и регулярные выражения

- [Про историю термина](https://habr.com/ru/articles/169765/)
- [Про регулярные выражения на Python](https://habr.com/ru/articles/349860/)

Регулярное выражение — это строка, задающая шаблон поиска подстрок в тексте

Иными словами, это такие строковые шаблоны, которые помогают найти подстроки определенного вида в строке

Для работы с регулярными выражениями в Python как правило используется встроенный модуль `re`(про альтернативы в конце)

Пример таких шаблонов:

| Регулярное выражение | Описание |
|----------------------|----------|
| simple text          | В точности текст «simple text» |
| \d{5}                | Последовательности из 5 цифр<br>\d означает любую цифру<br>{5} — ровно 5 раз |
| \d\d/\d\d/\d{4}      | Даты в формате ДД/ММ/ГГГГ<br>(и прочие куски, на них похожие, например, 98/76/5432) |
| \b\w{3}\b            | Слова в точности из трёх букв<br>\b означает границу слова (с одной стороны буква, а с другой — нет)<br>\w — любая буква,<br>{3} — ровно три раза |
| [-+]?\d+             | Целое число, например, 7, +17, -42, 0013 (возможны ведущие нули)<br>[-+]? — либо -, либо +, либо пусто<br>\d+ — последовательность из 1 или более цифр |


Давайте разбираться!

Будем рассматривать использование регулярных выражений (и в целом их синтаксис) на примере конструкции 
`re.search(<pattern>, <text>)`, где `<pattern>` - это регулярное выражение, а `<text>` - это текст, внутри которого мы ищем подстроки согласно нашему паттерну

Небольшая справка:


- Промежутки, заключенные в квадратные скобки, позволяют найти цифры или буквы разных алфавитов и разных регистров
    - [0-9] соответствует любой цифре
    - [A-Z] соответствует любой заглавной букве английского алфавита
    - [a-z] соответствует любой строчной букве английского алфавита
    - [А-Я] и [а-я] ‒ аналогично для букв русского алфавита (кроме буквы ё/Ё - ее нужно включать отдельно!)
- Для обозначения произвольного символа (кроме новой строки) используется точка ‒ .
- Для обозначения начала строки используется символ ^ (имеет другое значение внутри `[]`)
- Для обозначения конца строки используется символ $ 
- Для цифр есть специальный символ \d (от digit) ≈[0-9].. Добавление обратного слэша называется экранированием: так мы отмечаем, что ищем именно цифру, а не просто букву d.
- Для любого символа, кроме цифры тоже есть специальный символ \D (от digit)≈[^0-9] (заглавная буква здесь отвечает за отрицание). Добавление обратного слэша называется экранированием: так мы отмечаем, что ищем именно цифру, а не просто букву D.
- Для пробела тоже существует свой символ ‒ \s (от space) ≈[ \f\n\r\t\v]. Этот символ соответствует ровно одному пробельному символу в тексте (пробел, табуляция, перенос строки и т.д.).
- Любой непробельный символ, обозначается как \S (заглавная буква здесь отвечает за отрицание).
- Для букв тоже существует свой символ ‒ \w (от word) ≈ [0-9a-zA-Zа-яА-ЯёЁ_]. Любая буква (то, что может быть частью слова), а также цифры и _ .
- Любая не-буква, не-цифра и не подчёркивание, обозначается как \W (заглавная буква здесь отвечает за отрицание)

**Важное замечание**

Синтаксис регулярных выражений чувствителен к регистру. Т.е. если где-либо в регулярном выражении используются заглавные/строчные буквы, это имеет причину

In [229]:
import re

### Примеры по мотивам справки

In [363]:
text = "Here is some text to search inside"
pattern = "inside"

result = re.search(pattern, text)
print(result)
print(result.span())

<re.Match object; span=(28, 34), match='inside'>
(28, 34)


In [365]:
result = re.search("insidde", text)
print(result)
print(result.span())

None


AttributeError: 'NoneType' object has no attribute 'span'

Настоящая сила регулярных выражений раскрывается, когда в регулярном выражении (шаблоне) присутствуют **метасимволы**, т.е. символы, которые имеют особенное значение 

#### []

Например, квадратные скобки `[]` используются для поиска множества символов:

In [375]:
text = "Я до сих пор слышу это в голове... 88005553535... Проще позвонить, чем у кого-то занимать..."

pattern = "[0-9]" # будет произведен поиск одной любой цифры
print(re.search(pattern, text))
# А что если мы хотим больше одной?

<re.Match object; span=(35, 36), match='8'>


In [369]:
pattern = "[0-9][0-9]" # будет произведен поиск двух любых цифр
print(re.search(pattern, text))

<re.Match object; span=(35, 37), match='88'>


А что если мы хотим искать числа произвольной длины?

Забегая вперёд, для этого есть специальный символ:
- `+` - для поиска одного и более повторений

In [374]:
pattern = "[0-9]+" # будет произведен поиск одной и более цифры
print(re.search(pattern, text))

<re.Match object; span=(35, 40), match='88005'>


Символы в скобках необязательно должны быть в формате какого-то диапазона, как `[0-9]` или `[A-z]`, например:

In [377]:
pattern = "[бес][бес]" # будет произведен поиск одного из символов (б, е, с) по порядку
text = "мракобесие"

print(re.search(pattern, text))

<re.Match object; span=(5, 7), match='бе'>


In [381]:
# более осмысленный пример:
pattern = "[Ии]ван"  # поиск текста `Иван` или `иван`
text = "Скажи-ка мне, Ванюша! Скажи-ка мне, Иван!"

print(re.search(pattern, text))

<re.Match object; span=(36, 40), match='Иван'>


#### . (точка)

Символ точки (`.`) соответствует одному любому символу в строке. Как вы думаете, какие из строк ниже соответствуют приведенному шаблону?

In [ ]:
pattern = ".от"

texts = ["подушка", "Кот", "Живот", "крот", "от"]
for text in texts:
    result = re.search(pattern, text)
    if result is not None:
        print(f"Pattern {pattern} was found in {text}")
    else:
        print(f"Pattern {pattern} was not found in {text}")

#### ^ и $

Иногда нам хочется не просто найти какую-то подстроку внутри строки, но быть уверенными в том, что она находится:
- В самом начале слова (символ `^`)
- В самом конце слова (символ `$`). Символ `$` также позволит найти шаблон перед переносом строки (т.е. перед `\n`)
- Находится ровно между началом и концом (т.е. шаблон соответствует тексту целиком)


In [277]:
text = """
[Verse 1]
Buddy, you're a boy, make a big noise
Playing in the street
Gonna be a big man someday
You got mud on your face
You big disgrace
Kickin' your can all over the place, singing
""".strip()

pattern = "^\[Verse [0-9]\]"  # паттерн для поиска [Verse *цифра*] в начале строки 
re.search(pattern, text)

<re.Match object; span=(0, 9), match='[Verse 1]'>

In [280]:
text = """
[Chorus]
We will, we will rock you
We will, we will rock you
""".strip()

pattern = "you$"  # поиск you в самом конце строки [в некотором смысле похоже на str.rfind]
re.search(pattern, text)

<re.Match object; span=(57, 60), match='you'>

#### Синтаксис [^]

Символ `^` имеет другое значение, если внутри `[]`. Такая запись `[^...]` означает поиск такого символа, который не соответствует чему-либо внутри `[]` (то есть это по сути отрицание множества символов)

In [290]:
text = "Я уже совсем большой, мне 5 лет!"
pattern = "[^0-9]"  # поиск символа, который не соответствует никакой цифре

re.search(pattern, text)

<re.Match object; span=(0, 1), match='Я'>

#### Синтаксис \\*letter*

##### \\w и \\W

\w (от word) соответствует набору [0-9a-zA-Zа-яА-ЯёЁ_], т.е. это любая буква (то, что может быть частью слова), а также цифры и _ 

In [287]:
text = "Вы нас балуете, наша светлость"
pattern = "\w"

re.search(pattern, text)

<re.Match object; span=(0, 1), match='В'>

А какую подстроку мы найдем с помощью вот такого регулярного выражения?

In [ ]:
text = "Вы нас балуете, наша светлость"
pattern = "\w+"


print(re.search(pattern, text))

\W (с большой буквы) - это противоположность \w (с маленькой)

\W эквивалентно [^0-9a-zA-Zа-яА-ЯёЁ_] -> т.е. поиск любого символа, который не является цифрой, буквой алфавита или символом "_"

In [295]:
pattern = "\W"
text = "То-то же, будешь знать!"

re.search(pattern, text)

<re.Match object; span=(2, 3), match='-'>

##### \d и \D

- \d - аналог [0-9]
- \D - аналог [^0-9]

In [296]:
text = "Я до сих пор слышу это в голове... 88005553535... Проще позвонить, чем у кого-то занимать..."

pattern = "\d" # будет произведен поиск одной любой цифры
print(re.search(pattern, text))

<re.Match object; span=(35, 36), match='8'>


In [297]:
text = "Я до сих пор слышу это в голове... 88005553535... Проще позвонить, чем у кого-то занимать..."

pattern = "\D" # будет произведен поиск одного нечислового символа
print(re.search(pattern, text))

<re.Match object; span=(0, 1), match='Я'>


##### \s и \S 

- \s используется для поиска пробельных символов
- \S используется для поиска любых символов, кроме пробельных

In [302]:
bad_str_8bit = "Hello\xa0World!" 
bad_str_unicode = "Hello\u2008World!"
good_str = "Hello World!"

pattern = "\s"

print(re.search(pattern, bad_str_8bit))
print(re.search(pattern, bad_str_unicode))
print(re.search(pattern, good_str))

<re.Match object; span=(5, 6), match='\xa0'>
<re.Match object; span=(5, 6), match='\u2008'>
<re.Match object; span=(5, 6), match=' '>


#### Якори

Якори - это категория специальных символов внутри регулярных выражений, которые отвечают за положение искомого шаблона
`^` и `$` - примеры таких якорей, отвечающих за начало и конец строки соответственно

Особенностью якорей является то, что сами по себе они не соответствуют какому-либо символу (т.е. это zero-width match)

In [303]:
text = "Какой-то текст"
pattern = "^"

re.search(pattern, text)

<re.Match object; span=(0, 0), match=''>

##### \b и \B

- \b - указатель на начало слова или его конец
- \B - противоположен ему, т.е. указывает на не-начало слова или не-конец слова

Слово - это \w (т.е. [a-zA-Z0-9_])

In [310]:
text = 'И он всё тщательно пережевывал: жевал, жевал...'
result = re.search(r'\bжев', text)
print(result)
l, r = result.span()
print(text[l-5:l+5])

<re.Match object; span=(32, 35), match='жев'>
вал: жевал


In [312]:
result = re.search(r'жев', text)
print(result)
l, r = result.span()
print(text[l-5:l+5])

<re.Match object; span=(23, 26), match='жев'>
 пережевыв


In [314]:
text = 'И он всё жевал, жевал...Да пережевывал'
result = re.search(r'\Bжев', text)
print(result)
l, r = result.span()
print(text[l-5:l+5])

<re.Match object; span=(31, 34), match='жев'>
 пережевыв


#### Квантификаторы

Квантификаторы - это категория специальных символов, используемая для обозначения количества повторений шаблона

- `*` - ноль или более повторений
- `+` - одно или больше повторений
- `?` - ноль или одно повторение

Квантификаторы должны стоять после какого-то шаблона. Например, следующее выражение `.?` означает "ноль или один любой символ"

In [317]:
text = "8800553535...проще позвонить..."
pattern = "[0-9]+"  # одна или более цифра

re.search(pattern, text)

<re.Match object; span=(0, 10), match='8800553535'>

In [318]:
text = "8800553535...проще позвонить..."
pattern = "[0-9]?"  # одна или 0 цифр

re.search(pattern, text)

<re.Match object; span=(0, 1), match='8'>

In [320]:
text = "8800553535...проще позвонить..."
pattern = "[0-9]*п"  # 0 или более цифр, после которых идет буква "п"

re.search(pattern, text)

<re.Match object; span=(13, 14), match='п'>

__Важно__

По умолчанию квантификаторы производят поиск самого длинного шаблона из возможных внутри строки (т.е. они работают жадно)

Как вы думаете, какой результат вернет поиск по шаблону ниже?

In [ ]:
re.search('<.*>', '<div><span> <br> </span></div>')

Квантификаторы можно использовать в нежадном режиме, дописав к ним знак `?`:
- `*?`
- `+?`
- `??`

Тогда будет произведен поиск самого короткого шаблон:

In [ ]:
re.search('<.*?>', '<div><span> <br> </span></div>')

## Что почитать

__Про unicodedata__:

- [Категории символов unicode](https://www.unicode.org/L2/L1999/UnicodeData.html) (например, для `unicodedata.category`), раздел "General Category", это для справки
- [Про нормализацию юникода](https://habr.com/ru/articles/45489/). Здесь концептуально объясняется, в чем вообще принцип нормальных форм
- [Про нормализацию юникода, но на конкретном примере](https://stackoverflow.com/questions/16467479/normalizing-unicode). Здесь смотрите самый первый ответ (который ~137 апвоутов)
- [Документация модуля unicodedata](https://docs.python.org/3/library/unicodedata.html), для справки

__Про проблемы с html entities__:
- [Что такое html entities (и их примеры)](https://www.w3schools.com/html/html_entities.asp)
- [Документация модуля html в Python](https://docs.python.org/3/library/html.html), для справки

__Про регулярные выражения__:
- [Про историю возникновения](https://habr.com/ru/articles/169765/)
- Материал данной лекции (и семинаров по ней) сильно основан по логике рассказа на [этой статье](https://realpython.com/regex-python/) и её второй части. До семинара стоит её почитать до части [Grouping constructs and backreferences](https://realpython.com/regex-python/#grouping-constructs-and-backreferences) не включительно (т.е. про группировки пока читать не обязательно)
- [Вот этот ридинг про регулярные выражения](https://habr.com/ru/articles/349860/). Он содержит сильно больше материала, чем лекция, и не соответствует ей по логике рассказа, но внутри ридинга есть много разных задач на регулярные выражения + картинки и примеры, так что может быть полезно

__Про полезные модули/пакеты__

Как правило в Python с регулярными выражениями можно работать с помощью встроенного модуля `re`, но есть ещё:
- [regex](https://pypi.org/project/regex/) (быстрее, чем re, плюс есть дополнительные фичи)
- [re2](https://pypi.org/project/re2/) (быстрее, чем re и regex)